# Import Libraries

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import os
import tensorflow as tf
from tensorflow import keras
import cv2

# Get data from file

In [2]:
os.listdir()

['.ipynb_checkpoints', 'Data', 'LivenessDetection.ipynb']

In [3]:
file = open(r'C:\Users\Viet\Desktop\jupyter\ZaloAI\Data\train\label.csv')

In [4]:
dir = r'C:\Users\Viet\Desktop\jupyter\ZaloAI\Data\train\label.csv'

In [5]:
os.chdir(r'C:\Users\Viet\Desktop\jupyter\ZaloAI\Data')

In [6]:
train_df = pd.read_csv('train\label.csv')
train_df

,fname,liveness_score
0,1.mp4,0
1,2.mp4,1
2,3.mp4,1
3,5.mp4,0
4,7.mp4,1
...,...,...
1163,1880.mp4,1
1164,1881.mp4,0
1165,1882.mp4,0
1166,1884.mp4,0


In [7]:
dataset_path = os.listdir('test/videos')
all_rooms = []
for item in dataset_path:
    all_rooms.append((str(item),''))
test_df = pd.DataFrame(data=all_rooms, columns=['fname', 'liveness_score'])
test_df

,fname,liveness_score
0,0.mp4,
1,100.mp4,
2,1001.mp4,
3,1005.mp4,
4,1022.mp4,
...,...,...
345,972.mp4,
346,973.mp4,
347,979.mp4,
348,981.mp4,


In [8]:
import joblib
train_data = (joblib.load('train[0]res.sav'), joblib.load('train[1]res.sav'))
train_labels = joblib.load('train_labelsres.sav')
test_data = (joblib.load('test[0]res.sav'), joblib.load('test[1]res.sav'))

# Read a frame and extract features

In [8]:
IMG_SIZE = 224


def crop_center_square(frame):
    y, x = frame.shape[0:2]
    min_dim = min(y, x)
    start_x = (x // 2) - (min_dim // 2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]


def load_video(path, max_frames=0, resize=(IMG_SIZE, IMG_SIZE)):
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame = crop_center_square(frame)
            frame = cv2.resize(frame, resize)
            frame = frame[:, :, [2, 1, 0]]
            frames.append(frame)

            if len(frames) == max_frames:
                break
    finally:
        cap.release()
    return np.array(frames)

In [9]:
dir = os.getcwd()
# print(os.path.join(dir, 'train\\videos\\1.mp4'))
# frame_test = load_video(os.path.join(dir, 'train/videos/1.mp4'))
# frame_test

In [10]:
def build_feature_extractor():
    feature_extractor = keras.applications.ResNet50(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.resnet50.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()

In [11]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 100

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [12]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["fname"].values.tolist()
    
    ##take all classlabels from train_df column named 'tag' and store in labels
    labels = df["liveness_score"].values

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool") # 145,20
    frame_features = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32") #145,20,2048

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        frames = load_video(os.path.join(root_dir, path))
        frames = frames[None, ...]
        #if (len(frames) >= 1):
         #   print(1)
        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            #print(feature_extractor.input_shape)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train_df, "train/videos")
test_data, test_labels = prepare_all_videos(test_df, "test/videos")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")



print(f"train_labels in train set: {train_labels.shape}")

print(f"test_labels in train set: {test_labels.shape}")


1/1 [==============================] - 0s 146ms/step


1/1 [==============================] - 0s 172ms/step


1/1 [==============================] - 0s 243ms/step


1/1 [==============================] - 0s 137ms/step


1/1 [==============================] - 0s 192ms/step


...

1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step


1/1 [==============================] - 2s 2s/step
Frame features in train set: (1168, 20, 2048)
Frame masks in train set: (1168, 20)
train_labels in train set: (1168,)
test_labels in train set: (350,)


In [32]:
train_labels

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

# Training model

In [14]:
def get_sequence_model():
    #class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    x = keras.layers.LSTM(64, return_sequences=True)(frame_features_input, mask=mask_input)
    x = keras.layers.LSTM(64, return_sequences=False)(x)
    #x = keras.layers.LSTM(64, return_sequences=False)(x)
    #x = keras.layers.LSTM(32, return_sequences=False)(x)
    x = keras.layers.Dropout(0.4)(x)
    #x = keras.layers.GRU(8)(x)
    #x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(64, activation="relu")(x)
    x = keras.layers.Dense(32, activation="relu")(x)
    #x = keras.layers.Dense(16, activation="relu")(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(2, activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="SGD", metrics=["accuracy"]
    )
    return rnn_model

In [15]:
EPOCHS = 100
# Utility for running experiments.
def run_experiment():
    filepath = "./tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)

    return history, seq_model


_, sequence_model = run_experiment()

Epoch 1/100
25/26 [===========================>..] - ETA: 0s - loss: 0.6943 - accuracy: 0.4988
Epoch 1: val_loss improved from inf to 0.68476, saving model to ./tmp\video_classifier
26/26 [==============================] - 14s 183ms/step - loss: 0.6939 - accuracy: 0.5018 - val_loss: 0.6848 - val_accuracy: 0.5983
Epoch 2/100
25/26 [===========================>..] - ETA: 0s - loss: 0.6836 - accuracy: 0.5663
Epoch 2: val_loss improved from 0.68476 to 0.67208, saving model to ./tmp\video_classifier
26/26 [==============================] - 2s 62ms/step - loss: 0.6829 - accuracy: 0.5692 - val_loss: 0.6721 - val_accuracy: 0.6496
Epoch 3/100
25/26 [===========================>..] - ETA: 0s - loss: 0.6670 - accuracy: 0.6675
Epoch 3: val_loss improved from 0.67208 to 0.65236, saving model to ./tmp\video_classifier
26/26 [==============================] - 2s 59ms/step - loss: 0.6672 - accuracy: 0.6634 - val_loss: 0.6524 - val_accuracy: 0.6980
Epoch 4/100
25/26 [===========================>..] - E

Epoch 29/100
26/26 [==============================] - ETA: 0s - loss: 0.1902 - accuracy: 0.9327
Epoch 29: val_loss did not improve from 0.33264
26/26 [==============================] - 2s 63ms/step - loss: 0.1902 - accuracy: 0.9327 - val_loss: 0.3387 - val_accuracy: 0.8547
Epoch 30/100
26/26 [==============================] - ETA: 0s - loss: 0.2158 - accuracy: 0.9106
Epoch 30: val_loss improved from 0.33264 to 0.33013, saving model to ./tmp\video_classifier
26/26 [==============================] - 2s 65ms/step - loss: 0.2158 - accuracy: 0.9106 - val_loss: 0.3301 - val_accuracy: 0.8689
Epoch 31/100
25/26 [===========================>..] - ETA: 0s - loss: 0.2073 - accuracy: 0.9262
Epoch 31: val_loss did not improve from 0.33013
26/26 [==============================] - 2s 72ms/step - loss: 0.2037 - accuracy: 0.9278 - val_loss: 0.3421 - val_accuracy: 0.8519
Epoch 32/100
25/26 [===========================>..] - ETA: 0s - loss: 0.1312 - accuracy: 0.9525
Epoch 32: val_loss did not improve fro

Epoch 59/100
26/26 [==============================] - ETA: 0s - loss: 0.0053 - accuracy: 0.9988
Epoch 59: val_loss did not improve from 0.31800
26/26 [==============================] - 2s 71ms/step - loss: 0.0053 - accuracy: 0.9988 - val_loss: 0.5247 - val_accuracy: 0.8860
Epoch 60/100
25/26 [===========================>..] - ETA: 0s - loss: 0.0038 - accuracy: 1.0000
Epoch 60: val_loss did not improve from 0.31800
26/26 [==============================] - 2s 68ms/step - loss: 0.0038 - accuracy: 1.0000 - val_loss: 0.5229 - val_accuracy: 0.8832
Epoch 61/100
25/26 [===========================>..] - ETA: 0s - loss: 0.0057 - accuracy: 1.0000
Epoch 61: val_loss did not improve from 0.31800
26/26 [==============================] - 2s 67ms/step - loss: 0.0056 - accuracy: 1.0000 - val_loss: 0.5076 - val_accuracy: 0.8803
Epoch 62/100
25/26 [===========================>..] - ETA: 0s - loss: 0.0041 - accuracy: 1.0000
Epoch 62: val_loss did not improve from 0.31800
26/26 [===========================

Epoch 89/100
26/26 [==============================] - ETA: 0s - loss: 9.5006e-04 - accuracy: 1.0000
Epoch 89: val_loss did not improve from 0.31800
26/26 [==============================] - 1s 58ms/step - loss: 9.5006e-04 - accuracy: 1.0000 - val_loss: 0.6517 - val_accuracy: 0.8860
Epoch 90/100
25/26 [===========================>..] - ETA: 0s - loss: 8.7124e-04 - accuracy: 1.0000
Epoch 90: val_loss did not improve from 0.31800
26/26 [==============================] - 2s 61ms/step - loss: 8.6800e-04 - accuracy: 1.0000 - val_loss: 0.6478 - val_accuracy: 0.8889
Epoch 91/100
26/26 [==============================] - ETA: 0s - loss: 0.0010 - accuracy: 1.0000
Epoch 91: val_loss did not improve from 0.31800
26/26 [==============================] - 1s 58ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 0.6486 - val_accuracy: 0.8889
Epoch 92/100
25/26 [===========================>..] - ETA: 0s - loss: 9.3460e-04 - accuracy: 1.0000
Epoch 92: val_loss did not improve from 0.31800
26/26 [=======

In [24]:
sequence_model.save('resnet+lstm89,46.h5')

# Save data to file

In [15]:
import joblib
# Dumps into file
joblib.dump(train_data[0], 'train[0]res.sav')
# Loads from file
places = joblib.load('train[0]res.sav')
print(places)

[[[0.1561236  0.5042326  0.4307025  ... 0.82540977 0.26249433 0.740791  ]
  [0.16421844 0.45945486 0.47137842 ... 0.8598732  0.27059653 0.7810845 ]
  [0.17585237 0.5023016  0.4321668  ... 0.8901333  0.3397658  0.7984743 ]
  ...
  [0.2696781  0.39567894 0.49033448 ... 1.0964377  0.28182536 0.90030664]
  [0.1110843  0.41196927 0.6002062  ... 1.0777488  0.29943222 0.6897735 ]
  [0.12992406 0.39379618 0.5952001  ... 1.0741152  0.33618212 0.627323  ]]

 [[1.0034592  0.37644726 0.22494024 ... 1.8880258  1.0510021  0.8583887 ]
  [0.7004772  0.38692102 0.19517331 ... 1.4274206  0.7545161  1.0237236 ]
  [0.7281393  0.3334929  0.20475146 ... 1.3139347  1.0222927  1.1384487 ]
  ...
  [0.79146725 0.48806843 0.32986575 ... 2.0811617  0.34316772 1.6333749 ]
  [0.71079147 0.4505908  0.29848877 ... 1.9716486  0.42519125 1.3967702 ]
  [0.5834979  0.26977402 0.2654699  ... 1.9170332  0.48128664 1.6933436 ]]

 [[0.17595239 0.14846066 0.         ... 0.2063537  0.04364658 1.0430156 ]
  [0.16942838 0.148720

In [16]:
joblib.dump(train_data[1], 'train[1]res.sav')
# Loads from file
places = joblib.load('train[1]res.sav')
print(places)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [17]:
joblib.dump(test_data[0], 'test[0]res.sav')
# Loads from file
places1 = joblib.load('test[0]res.sav')
print(places)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [18]:
joblib.dump(test_data[1], 'test[1]res.sav')
# Loads from file
places1 = joblib.load('test[1]res.sav')
print(places)

[[ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 ...
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]]


In [19]:
train_labels

array([0, 1, 1, ..., 0, 0, 0], dtype=int64)

In [20]:
joblib.dump(train_labels, 'train_labelsres.sav')
# Loads from file
places1 = joblib.load('train_labelsres.sav')
print(places1)

[0 1 1 ... 0 0 0]


# Predict test

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1 

    return frame_features, frame_mask

In [ ]:
def sequence_prediction(path):
    frames = load_video(os.path.join("test/videos", path))
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]
    
    return probabilities

results = []
label = 0
all_test_video = test_df["fname"].values.tolist()
for test_video in all_test_video:
    predict = sequence_prediction(test_video)
    if (predict[0] >= 0.5):
        label = 0
    else:
        label = 1
    results.append((test_video, label))
    print(test_video, label)

In [16]:
predict = sequence_model.predict([test_data[0], test_data[1]])

11/11 [==============================] - 4s 21ms/step


In [17]:
len(predict)

350

In [18]:
liveness_score = []
for i in range(len(predict)):
    if predict[i][0] >= 0.5:
        label = 0
    else: 
        label = 1
    liveness_score.append(label)

In [19]:
test_df['liveness_score'] = liveness_score

In [20]:
test_df

,fname,liveness_score
0,0.mp4,0
1,100.mp4,1
2,1001.mp4,1
3,1005.mp4,1
4,1022.mp4,1
...,...,...
345,972.mp4,0
346,973.mp4,1
347,979.mp4,1
348,981.mp4,1


In [21]:
test_df.to_csv('PredictResNet50,2.csv', index=False)

# Print result

In [ ]:
results = list(set(results))
output = pd.DataFrame(data = results, columns =['fname', 'liveness_score'])
output.to_csv('Predict.csv', index=False)

In [ ]:
output[:10]